In [ ]:
from groq import Groq
import json
import csv

# System and user message settings
system_message = """You are a helpful assistant that generates pairs of Persian date representations (informal and formal).
You are familiar with the Jalali calendar months, where Farvardin is the first month.
Please generate dates accurately with diverse formats."""

user_message = """Please generate 2000 pairs of dates in the following formats:

1. Informal date in Persian (with variation in representation)
2. The same date in the standard format "YYYY/MM/DD"

For informal dates, use different styles, such as:
- Regular Persian order: "1 مهر 1403" or "اول مهر سال 1403"
- Numeric order: "1 1403 مهر"
- Year mentioned first: "سال مهر اول 1403"
- Using dashes: "1403-07-01"

For formal dates, always use the "YYYY/MM/DD" format, even if the input uses dashes.

Please use various Jalali months and different days. Randomly select dates and use different years (between 1300 and 1400).

Please provide the output as a list of date pairs in JSON format. Example:

[
  {"informal": "1403-07-01", "formal": "1403/07/01"},
  {"informal": "پنجم اردیبهشت 1402", "formal": "1402/02/05"},
  {"informal": "17 آبان 1395", "formal": "1395/08/17"},
  {"informal": "1 1403 مهر", "formal": "1403/07/01"},
  {"informal": "سال مهر اول 1403", "formal": "1403/07/01"},
  ...
]
Only provide the JSON file without any additional text.
Generate the output as a JSON file."""

client = Groq(api_key="<API KEY>")

def preprocess_output(output):
    # Find the start and end of the JSON array
    last_index = output.rindex('informal')
    start = output.find('[')
    list_str = output[start:last_index-6]+']'
    return list_str

csv_filename = 'date.csv'

for i in range(200):
    print(f"Execution number {i+1}")

    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0.4,
        max_tokens=4000,
        top_p=1,
        stream=False,
        stop=None,
    )

    output = completion.choices[0].message.content.strip()
    processed_output = preprocess_output(output)

    if processed_output:
        try:
            # Convert the output to JSON
            date_pairs = json.loads(processed_output)
            print(f"Successfully converted to JSON. Number of date pairs: {len(date_pairs)}")
            
            # Save results to a CSV file in append mode
            with open(csv_filename, 'a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                
                for pair in date_pairs:
                    writer.writerow([pair['informal'], pair['formal']])
            
            print(f"Data saved to {csv_filename}.")
        except json.JSONDecodeError as e:
            print(f"Error converting to JSON: {e}")
            print("Please review the preprocessed output:")
            print(processed_output)
    else:
        print("Error in preprocessing output. JSON array not found.")
